## This notebook shows how you can use the `fastplotlib` API to link `Graphic` objects together

In [1]:
import numpy as np
from fastplotlib.layouts import GridPlot
from fastplotlib.graphics import *
from fastplotlib import Plot
from ipywidgets import IntSlider, VBox, Layout
from fastplotlib.graphics.line_slider import LineSlider
from tifffile import imread

In [2]:
# load in contours and temporal components as np arrays
contours = np.load("/home/caitlin/repos/CCN_caiman_mesmerize_workshop_2023/fpl-demos/data/contours.npy", allow_pickle=True)
temporal = np.load("/home/caitlin/repos/CCN_caiman_mesmerize_workshop_2023/fpl-demos/data/temporal.npy", allow_pickle=True)

# load in Sue demo movie, may need to change paths
input_movie = imread("/home/caitlin/caiman_data/example_movies/Sue_2x_3000_40_-46.tif")

In [3]:
# generating random colors 
rand_colors = np.random.rand(len(contours), 4)  # [n_contours, RGBA]
rand_colors[:, -1] = 1 # set alpha = 1

In [4]:
# creating graphics
# image graphic for input_movie
movie_graphic = ImageGraphic(data=input_movie[0], cmap="gnuplot2")
# line collection for contours and temporal
contour_graphic = LineCollection(data=contours, colors=rand_colors)
temporal_graphic = LineCollection(data=temporal, colors=rand_colors)
# line stack for temporal
temporal_stack = LineStack(data=temporal, colors=rand_colors)

In [5]:
# create a gridplot to view the contours over the input movie and the temporal components as a stack
grid_plot = GridPlot((1, 2),  
                     names=[["contours", "temporal"]])

# can add graphics to subplots by name or position 
grid_plot["contours"].add_graphic(movie_graphic)
grid_plot[0, 0].add_graphic(contour_graphic)
grid_plot["temporal"].add_graphic(temporal_stack)

# create a plot to individually view the temporal components
plot = Plot()
plot.add_graphic(temporal_graphic)

# dynamically move through input movie
slider = IntSlider(min=0, max=input_movie.shape[0] - 1, value=0, step=1)

# vertical line slider to accompany the individual viewing of temporal components over time
_ls = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max()), slider=slider)
plot.add_graphic(_ls)

# vertical line slider to accompany the temporal line stack
_ls2 = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max() + temporal_stack.graphics[-1].position.y), slider=slider)
grid_plot["temporal"].add_graphic(_ls2)

# function to update each frame
def update_frame(change):
    ix = change["new"]
    movie_graphic.data = input_movie[ix]
    
slider.observe(update_frame, "value")

@plot.renderer.add_event_handler("resize")
def update_slider_width(*args):
    width, h = plot.renderer.logical_size
    slider.layout = Layout(width=f"{width}px")

VBox([grid_plot.show(), plot.show(), slider])

RFBOutputContext()

RFBOutputContext()

### autoscaling

In [6]:
plot.auto_scale()
plot.camera.scale.x = 0.85
grid_plot["temporal"].auto_scale()

In [7]:
# not important to understand 
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target._set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

### interactivity

In [8]:
# so we can view the temporal graphics one-by-one, first hide all of them
temporal_graphic[:].present = False

# link image to contours
movie_graphic.link(
    "click",
    target=contour_graphic,
    feature="colors", 
    new_data="w", 
    callback=euclidean
)

# contours colors -> contour thickness
contour_graphic.link("colors", target=contour_graphic, feature="thickness", new_data=5)

# temporal stack events 
temporal_stack.link("click", target=temporal_stack, feature="colors", new_data="w")
temporal_stack.link("colors", target=temporal_stack, feature="thickness", new_data=4)

# contours <-> temporal stack
contour_graphic.link("colors", target=temporal_stack, feature="colors", new_data="w", bidirectional=True)

# temporal stack -> temporal single
temporal_stack.link("colors", target=temporal_graphic, feature="present", new_data=True)

# autoscale temporal plot to the current temporal component
temporal_graphic[:].present.add_event_handler(plot.auto_scale)

/home/caitlin/repos/fastplotlib/fastplotlib/graphics/features/_base.py:87: UserWarning: Event handler <bound method Interaction.event_handler of LineCollection @ 0x7f599018cfa0
Collection of <71> Graphics> is already registered.
  warn(f"Event handler {handler} is already registered.")
/home/caitlin/repos/fastplotlib/fastplotlib/graphics/features/_base.py:87: UserWarning: Event handler <bound method Interaction.event_handler of LineStack @ 0x7f599012d850
Collection of <71> Graphics> is already registered.
  warn(f"Event handler {handler} is already registered.")


In [ ]:
# make sure to close plots and kill kernel
plot.canvas.close()
grid_plot.canvas.close()